In [3]:
import copy
import numpy as np
import pandas as pd
import dask.array as da
import dask.bag as db
import dask.dataframe as dd
from dask import delayed, compute
from dask.datasets import timeseries
# import tc_distributed as tcd
import tc_distributed_pro as tcdp
import transforms_distributed as tfd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from boruta import BorutaPy

una_oprs = tfd.unary_operators
bina_oprs = tfd.binary_operators
oprs_list = [una_oprs, bina_oprs]

### 对代码的一系列简化测试

In [17]:
# dataset = 'data/php0iVrYT.csv'
# dat = tcdp.load_data(dataset, art='C')
dat = pd.read_csv('result/curr_php0iVrYT.csv')
dat.drop(dat.columns[0], axis=1, inplace=True)
print(dat)
x = dat.iloc[:, :-1].values
y = dat.iloc[:, -1].values.ravel()
feature_names = dat.columns[:-1]
model = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)
boruta_selector = BorutaPy(model, n_estimators = 'auto', verbose=0, random_state=1)
boruta_selector.fit(x, y)
supp = boruta_selector.support_
boruta_selector.ranking_
# boruta_ranking = boruta_selector.ranking_
# boruta_ranking
# selected_features = np.array(feature_names)[boruta_ranking <= 10]
# selected_features.shape
X_filtered = boruta_selector.transform(x)
df = pd.DataFrame(X_filtered, columns=dat.iloc[:, :-1].columns[supp], index=dat.index)
df['target'] = dat.iloc[:, -1]
print(df)

      (((V3_ln-V1_zscore)+(V3_ln-V4_zscore))+((V1+V2)_cos*(V1_cos+(V2/V4))))_adde  \
0                                             18.256592                             
1                                             17.209616                             
2                                             16.915390                             
3                                             13.675027                             
4                                             16.301434                             
...                                                 ...                             
1135                                          17.633575                             
1136                                          18.994880                             
1137                                          18.647648                             
1138                                          15.802334                             
1139                                          19.459675          

NameError: name 'supp' is not defined

In [ ]:
# 导入数据：这里刚读入的数据里都有第一列的一列索引，要把他删除掉
cur_dat = pd.read_csv('result/curr_php0iVrYT.csv')
cur_dat.drop(cur_dat.columns[0], axis=1, inplace=True)
cur_gen = pd.DataFrame(cur_dat.iloc[:, :-1], columns=cur_dat.columns[:-1])
print(cur_gen)
prev_dat = pd.read_csv('result/prev_php0iVrYT.csv')
prev_gen = prev_dat.drop(prev_dat.columns[0], axis=1, inplace=False)
prev_gen

In [ ]:
# 通过调用操作符生成子代特征
cands_bag = tcd.new_cands(cur_dat, prev_gen)
cands_bag = compute(compute(cands_bag)[0])[0]
cands_bag

In [ ]:
%%time
# 2021/1/31改进后完成只用3min 48s
# 生成下一代经历了竞争灭绝环境限制的子特征
next_g = tcd.getNextGen(cur_gen, prev_gen)
next_g.to_csv('test.csv')
next_g

In [ ]:
# 生成某个计算对应的自带特征
lis = []
# opr = oprs_list[0].get('kernelapproxrbf')
# candidates = delayed(opr()._exec)(cur_gen)
opr = oprs_list[1].get('minus')
candidates = delayed(opr()._exec)(cur_gen, prev_gen)
lis.append(candidates)
cands_bag = db.from_sequence(lis, partition_size=None, npartitions=None)
# cands_bag = tcd.new_cands(dat, prev_gen=prev_gen)
cands_bag = compute(compute(cands_bag)[0])[0]
cands_bag

In [ ]:
# 调用写好的函数直接生成子特征
# cands_lis = tcd.new_cands(cur_dat, prev_gen=prev_gen)

cands_bag = []
# cands_bag = tcd.new_cands(dat, prev_gen)
# print(compute(compute(cands_bag)[0])[0])
for oprs in oprs_list:
     for opr in oprs.values():
          candidates = None
          if oprs == oprs_list[0]:
               candidates = delayed(opr()._exec)(cur_gen)
          elif oprs == oprs_list[1]:
               candidates = delayed(opr()._exec)(cur_gen, prev_gen)
          cands_bag.append(candidates)
          # candidates_list.append(candidates.compute())
cands_bag = compute(compute(cands_bag)[0])[0]
print(cands_bag)
# print(compute(candidates_list)[0])

In [ ]:
cands_lis = []
for cand in cands_bag:
    cand = delayed(tcd.clean_dat)(cand)
    cands_lis.append(cand)
cands_lis = compute(cands_lis)[0]
cands_lis

In [ ]:
sorted_lis = []
for cand in cands_lis:
    if cand.empty:
        continue
    cand_dat = copy.deepcopy(cand)
    cand_dat[cur_dat.columns[-1]] = cur_dat.iloc[:, -1]
    sorted_gps = tcd.tribesCompetition(cand_dat)
    # tC 返回的结果是delayed格式的
    sorted_lis.append(sorted_gps)
sorted_lis = compute(sorted_lis)[0]
# sorted_lis = compute(compute(sorted_lis)[0])[0]
sorted_lis

In [ ]:
res_lis = []
# for i in range(len(sorted_lis)):
for i in range(4):
    cand_dat = copy.deepcopy(cands_lis[i-4])
    cand_dat[cur_dat.columns[-1]] = cur_dat.iloc[:, -1]
    cand_d = delayed(tcd.plunge_eliminate)(cand_dat, sorted_lis[i-4])
    res_lis.append(cand_d)
res_lis = compute(res_lis)[0]
res_lis

In [ ]:
# 和上一个cell功能类似,此cell只用于生成某个特定operator下的自带特征
res_lis = []
cand = cands_lis[-3]
cand_dat = copy.deepcopy(cand)
cand_dat[cur_dat.columns[-1]] = cur_dat.iloc[:, -1]
sorted_gps = tcd.tribesCompetition(cand_dat)
# tC 返回的结果是delayed格式的
sorted_gps = compute(sorted_gps)[0]
cand_d = delayed(tcd.plunge_eliminate)(cand_dat, sorted_gps)
res_lis.append(cand_d)

res_lis = compute(res_lis)[0]
res_lis

In [ ]:
from distributed import progress
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, f1_score
cand = cands_lis[-4]
d = delayed(train_test_split)(cand, cur_dat.iloc[:, -1])
X_train, X_test, y_train, y_test = compute(d)[0]
s = tcd.eval(X_train, X_test, y_train, y_test)
s = s.compute()
s

In [ ]:
%%time
from sklearn.model_selection import train_test_split

# r = delayed(train_test_split)(cand, cur_dat.iloc[:, -1])
r = train_test_split(cand, cur_dat.iloc[:, -1])
X_train, X_test, y_train, y_test = compute(r)[0]
# X_train, X_test, y_train, y_test
s = tcd.eval(X_train, X_test, y_train, y_test)[0]
compute(s)[0]

### 其余一些小的测试

In [ ]:
import numpy as np
lis = [0, 1, 2, 3, 4, 5, 6]

ind_lis = ()
for i in range(7):
    if lis[i] % 2:
        ind_lis = ind_lis + (i,)
lis = np.delete(lis, ind_lis).tolist()
lis

In [ ]:
dataset = 'data/php0iVrYT.csv'
dat = pd.read_csv(dataset)
res = pd.DataFrame()
N = 1
ddat = dd.from_pandas(dat, npartitions=N)
for col in dat.columns:
    if (dat[col].values > 0).any():
        res[str(col)+ '_abs'] = ddat[col].map(lambda x: x + np.e)
res

In [ ]:
@delayed
def add(x, y=1):
    return x + y

b = db.from_sequence(range(6), npartitions=2)
b = b.map(add, y=2)
b.compute()[0].compute()

### Setting up a cluster

In [ ]:
"""  """
from dask.distributed import LocalCluster, Client, progress
cluster = LocalCluster(n_workers=4, threads_per_worker=2, memory_limit='2GB')
client = Client(cluster)
# client = Client()
client
# client.ncores()

In [ ]:
client.shutdown()

In [ ]:
%%time
from time import sleep
def inc(x):
    sleep(1)
    return x + 1
def add(x, y):
    sleep(1)
    return x + y
# This takes three seconds to run because we call each function sequentially, one after the other
x1 = inc(1)
y1 = inc(2)
z1 = add(x1, y1)
z1

### Parallelize code with dask.delayed

In [ ]:
%%time
from dask import delayed
# This runs immediately, all it does is build a graph
x = delayed(inc)(1)
y = delayed(inc)(2)
z = delayed(add)(x, y)
# To get the result, call compute. Notice that this runs faster than the original code.
z.compute()

In [ ]:
# Look at the task graph for `z`
import graphviz
z.visualize()
# "dot -c" to be run in admin mode

### Parallelize a for-loop

In [ ]:
%%time
results = []
data = [1, 2, 3, 4, 5, 6, 7, 8]
for x in data:
    y = delayed(inc)(x)
    results.append(y)
total = delayed(sum)(results)
print("Before computing:", total) 
print("After computing :", total.compute())
total.visualize()

In [ ]:
import tc_distributed as tcd
import dask
dataset = 'data/php0iVrYT.csv'
dat = tcd.load_data(dataset, art='C')
lis = tcd.tribesCompetition(dat)
dask.compute(lis)

In [ ]:
from dask import delayed
from dask import compute
def fun(x):
    sums = []
    for i in range(x):
        sums.append(delayed(lambda n: n+1)(i))
    res = delayed(sum)(sums)
    return res.compute()
r = delayed(fun)(5)
# r.compute()
r.visualize()

In [ ]:
%%time
import dask.bag as db
import time
@delayed
def add(x, y):
    time.sleep(1)
    return x+y
lis = []
for i in range(3):
    cand = delayed(lambda x, y: add(x, y))(i+1, i+2)
    lis.append(cand)
print(lis)
# type(compute(lis))
b = db.from_sequence(lis)
# b = compute(compute(b)[0])[0]
b = compute(compute(compute(b)[0])[0])[0]
b
# type(b)
# lis = compute(lis)[0]
# lis

In [ ]:
%%time
import time
def add(x, y):
    time.sleep(1)
    return x+y
lis = []
for i in range(3):
    lis.append(add(i+1, i+2))
lis

In [ ]:
import pandas as pd
import numpy as np
from dask import compute, delayed
import tc_distributed as tcd
import dask.bag as db

dataset = 'data/php0iVrYT.csv'
dat = pd.read_csv(dataset)
print(dat)
seq = []
# res = pd.DataFrame()
d = []
for col in dat.columns:
    seq.append(str(col)+'_abs')
    d.append(delayed()(-dat[col]))
d = compute(d)[0]
d = np.transpose(np.array(d, dtype='float32'))
res = pd.DataFrame(d, columns=seq)
res
# for i in range(len(seq)):
#     res[str(seq[i])+ '_abs'] = res_lis[i]

# b = db.from_sequence(dat_lis)
# b = b.map(abs)
# b.compute()

In [ ]:
""" Often we want to delay only some functions, running a few of them immediately. 
    To delay or not to delay, is usually where we need to be thoughtful when using dask.delayed """
# %%time
def inc(x):
    sleep(1)
    return x + 1, 1

def double(x):
    sleep(1)
    return 2 * x

def is_even(x):
    return not x % 2
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
res = []
for i in range(2):
    for x in data:
        if is_even(x):
            y = delayed(double)(x)
        else:
            # continue
            y = delayed(inc)(x)[0]
        res.append(y)
total = delayed(sum)(res)
total2 = total / 5
%time total2.compute()
total2.visualize()

In [ ]:
from dask import delayed
from dask import compute   #  When having multiple outputs, use the dask.compute
x = delayed(np.arange)(10)
y = x ** 2
min_, max_ = compute(y.min(), y.max())
min_, max_

In [ ]:
%%time
dataset = 'data/php0iVrYT.csv'
df = delayed(pd.read_csv)(dataset)
sums = []
counts = []

by_class = df.groupby('Class')
total = by_class.V1.sum()
count = by_class.V1.count()
sums.append(total)
counts.append(count)
sums, counts = compute(sums, counts)
total_delays = sum(sums)
n_flights = sum(counts)
mean = total_delays / n_flights
mean

### 用装饰器完成delayed功能

In [ ]:
from dask import delayed
@delayed
def inc(x):
    return x + 1
@delayed
def add(x, y):
    return x + y

x = inc(15)
y = inc(30)
total = add(x, y)
print("total: ", total.compute())
# total.visualize()
dict(total.dask)

### Dask Distributed, Advanced

In [ ]:
def inc(x):
    return x + 1
# fut = client.submit(inc)
fut = client.submit(inc, 1)
fut

In [ ]:
fut

In [ ]:
from dask.distributed import wait, progress
progress(fut)
wait(fut)

In [ ]:
# grab the information back - this blocks if fut is not ready
client.gather(fut)
# equivalent action when only considering a single future
# fut.result()

In [ ]:
""" Here we see an alternative way to execute work on the cluster: when you submit or map with the inputs as futures, the computation moves to the data rather than the other way around, and the client, in the local Python session, need never see the intermediate values. This is similar to building the graph using delayed, and indeed, delayed can be used in conjunction with futures. """
# Some trivial work that takes time
# repeated from the Distributed chapter.

from dask import delayed
import time

def inc(x):
    time.sleep(5)
    return x + 1

def dec(x):
    time.sleep(3)
    return x - 1

def add(x, y):
    time.sleep(7)
    return x + y

In [ ]:
x = delayed(inc)(1)
y = delayed(dec)(2)
total = delayed(add)(x, y)
# notice the difference from total.compute()
# notice that this cell completes immediately
fut = client.compute(total)
fut

In [ ]:
client.gather(fut) # waits until result is ready

In [ ]:
x = client.submit(inc, 1)
y = client.submit(dec, 2)
total = client.submit(add, x, y)

print(total)     # This is still a future
total = client.persist(total)
client.gather(total)   # This blocks until the computation has finished

### background execution

In [ ]:
def add(x, y):
    return x + y

# Sometimes we defer computations with strings
x = 15
y = 30
z = "add(x, y)"
eval(z)



In [ ]:
def gen():
    res = 20
    yield res
    res += y
    yield res

next(gen())

### dask.bag = map, filter, toolz + parallel execute

In [ ]:
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(10)

In [ ]:
""" dask.bag manipulation """
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
# Operations on Bag objects create new bags.
c = b.filter(is_even).map(lambda x: x ** 2)
c
""" Call the .compute() method to trigger execution, as we saw for Delayed objects. """
c.compute()

In [ ]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

In [ ]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()


In [ ]:
# 同样功能
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()
b.foldby(lambda x: x % 2, binop=max, combine=max).compute()

### dask.array

In [ ]:

x = da.random.normal(10, 0.1, size=(20000, 20000),   # 400 million element array 
                              chunks=(1000, 1000))   # Cut into 1000x1000 sized chunks
x

In [ ]:
y = x.mean(axis=0)[::100]
print(x.nbytes/1e9, 'Gigabytes of the input processed lazily')
# y
%time y.compute()     # Time to compute the result
""" Notice that the Dask array computation ran in 4 seconds, but used 29.4 seconds of user CPU time. 
    The numpy computation ran in 19.7 seconds and used 19.6 seconds of user CPU time.
    Dask finished faster, but used more total CPU time 
    because Dask was able to transparently parallelize the computation because of the chunk size. 
"""

In [ ]:
dataset = 'data/php0iVrYT.csv'
df = pd.read_csv(dataset)
a = [da.from_array(df[col], chunks=(374,)) for col in df.columns]
a

In [ ]:
ddf = dd.read_csv(dataset)
col = df.columns[0]
# ddf.V1.nunique().compute()
ddf[col].nunique().compute()

In [ ]:
""" Stack this list of dask.array objects into a single dask.array object with da.stack """
aa = da.stack(a)
aa

### dask.dataframe

In [ ]:
dataset = 'data/php0iVrYT.csv'
ddf = dd.read_csv(dataset)
print(ddf)
ddf.head()

In [ ]:
""" Web Interface """
ddf = client.persist(ddf)
progress(ddf)

print(len(ddf))
ddf.count().compute()

In [ ]:
print((ddf.V1 == 0).sum().compute())
print(ddf.V1.sum().compute())
ddf.V1.mean().compute()

In [ ]:
ddf.groupby('Class').V1.mean().compute()

In [ ]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()

In [ ]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

In [ ]:
print(ddf.info())
sub_V2 = ddf[ddf.V2 >= 25]
sub_V2.shape[0].compute()

In [ ]:
cols = ['V1', 'V3', 'Class']
new_ddf = ddf[cols]
new_ddf = client.persist(new_ddf)
progress(new_ddf)
new_ddf.head()

In [ ]:
import multiprocessing
cpu_count()

In [ ]:
dataset = 'data/php0iVrYT.csv'
df = pd.read_csv(dataset)
a = [da.from_array(df[col], chunks=(374,)) for col in df.columns]
""" Stack this list of dask.array objects into a single dask.array object with da.stack """
aa = da.stack(a)
print(aa)

In [ ]:
N = 5
ddf = dd.from_pandas(df, npartitions=N)
""" Subset dask dataframe by column position """
cols = list(df.columns[2:4])
ddf[cols].compute()